**BİL - 210 FİNAL PROJESİ ÖDEVİ**

1906010 - Murat Cem Erenoğlu | 190601068 - Utku Dişci

Bu çalışmamızda bir alışveriş merkezinde teknolojik ürünler satan mağazaya ait tablet ürünlerinin özellikleri ve fiyatlarının bulunduğu bir veri seti üzerinde keşifçi veri analizi ve basit bir makine öğrenmesi algoritması geliştirmekteyiz.

**İÇİNDEKİLER**
1. KÜTÜPHANELERİ EKLEME
2. E.D.A
3. GÖRSELLEŞTİRME
4. DÜZENLEMELER
5. TRAINING & MACHINE LEARNING

# **1- KÜTÜPHANELERİ EKLEME**

Öncelikle gerekli kütüphanelerimizi ekliyoruz.

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import plotly as py
import matplotlib.pyplot as plt
from sklearn import preprocessing as sklpre
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score, f1_score, precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, auc, roc_curve, classification_report

from sklearn.linear_model import LinearRegression

import os

Verisetimizi df değişkenine atıyoruz.

In [ ]:
df = pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")

Şimdi sütunlarımızın adlarını Camel Style'a uyacak şekilde değiştiriyoruz.

In [ ]:
df = df.rename(columns = {"BataryaGucu":"bataryaGucu","Bluetooth":"bluetooth","MikroislemciHizi":"mikroislemciHizi","CiftHat":"ciftHat","OnKameraMP":"onKameraMP","DahiliBellek":"dahiliBellek","Kalinlik":"kalinlik","Agirlik":"agirlik","CekirdekSayisi":"cekirdekSayisi","ArkaKameraMP":"arkaKameraMP","CozunurlukYükseklik":"cozunurlukYukseklik","CozunurlukGenislik":"cozunurlukGenislik","BataryaOmru":"bataryaOmru","Dokunmatik":"dokunmatik","FiyatAraligi":"fiyatAraligi","Renk":"renk","Çok Ucuz":"cokUcuz"})

# **2- KEŞİFÇİ VERİ ANALİZİ**

Verisetimizin ilk 5 gözlemini görüntüleyelim.


In [ ]:
df.head(5)

Veri setimizin kaç gözlem ve öznitelikten oluştuğunu görüntülemek için shape kullanalım.

In [ ]:
df.shape

Veri çerçevesindeki değişkenlerin hangi tipte olduğunu ve bellek kullanımını görüntüleyelim.

In [ ]:
df.info()

Buradan anlıyoruz ki veri setimizde 20 farklı değişkenimiz var. Bunlardan dördü float64, sekizi int64 ve skizi de object tipinde.

Verisetimizde ki eksik verileri kontrol edelim.

In [ ]:
df.isna().sum()

5 satırda Ön kamera ve 12 satırda ise RAM bilgisinin eksik olduğunu görüyoruz. Bu eksiklikleri uygun bir şekilde doldurmak için birazdan gerekli işlemleri yapacağız.

Veri çerçevesinin temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T

# **3-GÖRSELLEŞTİRME**

Verisetimizde ki 'Renk' verisinin ne kadar dengeli dağıldığını gözlemleyelim

In [ ]:
df["renk"].value_counts()

Cihaz renklerinin dağılımını daha iyi inceleyebilmek için uygun renklerle bir pie chart çizdirelim.

In [ ]:
pie = df.renk.value_counts()
labels = pie.index
colors = ["ghostwhite","black","mediumorchid","goldenrod","tomato","lightpink","turquoise","saddlebrown","lemonchiffon","springgreen","gainsboro","royalblue"]
explode = [0,0,0,0,0,0,0,0,0,0,0,0]
print(len(explode))
sizes = pie.values

plt.figure(figsize = (8,8))
plt.pie(sizes, explode = explode, labels = labels, colors = colors, autopct = '%1.1f%%')
plt.show()

Korelasyon matrisini çizdirelim.

In [ ]:
df.corr()

Matrisimizde yanlışlıklar olmasını önlemek için değerlerimizdeki boş kısımları ortalamalar ile dolduralım.

In [ ]:
MeanRam=df['RAM'].mean()
df['RAM'] = df['RAM'].replace(np.nan, MeanRam)
MeanFrontCam=df['onKameraMP'].mean()
df['onKameraMP'] = df['onKameraMP'].replace(np.nan, MeanFrontCam)

df.isna().sum()

Görüldüğü üzere artık veri setimizde boş veri yok.

Şimdi daha rahat okunabilirlik için korelasyon matrisimizin ısı haritasını çizdirelim.

In [ ]:
sns.heatmap(df.corr(), cmap="rocket_r", vmin= 0.5, vmax=1.25,
           linewidth=0.3, cbar_kws={"shrink": .8})
plt.show()

Şimdi veri setimizi daha kolay anlayabilmek için bazı görselleştirmeler yapalım.

In [ ]:
sns.countplot(x = "bluetooth", data = df)

Bluetooth özelliğine sahip olan ve olmayan cihazların sayısının eşit olduğunu görüyoruz.

In [ ]:
sns.countplot(x = "fiyatAraligi", palette=['yellow',"red","blue","green"], data = df)

Cihazların fiyat aralıklarının 4 farklı kategoriye eşit bir şekilde dağıldıklarını gözlemleyebiliriz.

Dahili bellek değişkenine ait benzersiz değerleri görüntüleyelim.

In [ ]:
df["dahiliBellek"].unique()

Şimdi de mikroişlemci hızının cihazlar arasındaki dağılımını gözlemleyelim.

In [ ]:
plt.figure(figsize=(12,5))
ax = sns.countplot(x ='mikroislemciHizi', data = df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_xticklabels(ax.get_xticklabels(), fontsize=8)
plt.tight_layout()
plt.show()

Cihazların büyük bir çoğunluğunun 0.5 GHz mikroişlemci kullandığını bu tablo sayesinde gözlemleyebiliriz.

'RAM' ve 'Ağırlık' verilerimizi görselleştirelim.

In [ ]:
sns.boxplot(df["RAM"],color="salmon")

In [ ]:
sns.violinplot(df["agirlik"],color="salmon")

'Fiyat aralığı' ile 'Ram', 'Ağırlık' ve 'Arka Kamera MP' verilerinin arasında ki ilişkiyi gözlemleyelim.

In [ ]:
sns.relplot(x="fiyatAraligi", y="RAM", kind="line", ci="sd",color="red", data=df)

Bu çizgi grafik sayesinde RAM miktarının telefonun fiyatını direkt olarak etkilediğini görebiliriz.

In [ ]:
sns.relplot(x="fiyatAraligi", y="agirlik", kind="line", ci="sd",color="red", data=df)

In [ ]:
sns.relplot(x="fiyatAraligi", y="arkaKameraMP", kind="line", ci="sd",color="red", data=df)

Bu iki grafik bize fiyat aralıklarıyla ağırlığın ya da arka kamera kalitesinin ilişkisinin, fiyat/RAM ilişkisinden daha az etkili olduğunu gösteriyor.

Mikroişlemci ile batarya ömrü arasındaki ilişkiyi gözlemleyelim.

In [ ]:
sns.jointplot(x = df["bataryaOmru"], y = df["mikroislemciHizi"], kind = "kde", color = "salmon")

Şimdi de çekirdek sayısı ile mikroişlemci hızı arasındaki ilişkiyi gözlemleyelim.

In [ ]:
sns.jointplot(x = df["cekirdekSayisi"], y = df["mikroislemciHizi"], kind = "kde", color = "springgreen")

# **4-DÜZENLEMELER**

String verilerimizi integer verilere dönüştürüyoruz.

In [ ]:
labelEncoder = sklpre.LabelEncoder()

In [ ]:
df['WiFi'] = labelEncoder.fit_transform(df['WiFi'])
df['dokunmatik'] = labelEncoder.fit_transform(df['dokunmatik'])
df['ciftHat'] = labelEncoder.fit_transform(df['ciftHat'])
df['bluetooth'] = labelEncoder.fit_transform(df['bluetooth'])
df['3G'] = labelEncoder.fit_transform(df['3G'])
df['4G'] = labelEncoder.fit_transform(df['4G'])

In [ ]:
df.head()

Renk verilerini siliyoruz.

In [ ]:
df.drop('renk', inplace=True, axis=1)

In [ ]:
df.head(5)

# **5- TRAINING & MACHINE LEARNING**

Verilerimizi X (bağımsız değişkenler) ve y (bağımlı değişkenler) olarak ayırıyoruz.

In [ ]:
X = df.drop("fiyatAraligi", axis = 1)
y = df["fiyatAraligi"] 

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.15,random_state=42)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

GaussianNB Modeli İle Eğitim

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)

In [ ]:
y_pred = nb_model.predict(X_test)
y_pred

GaussianNB eğitim modelinin sonuçları:

In [ ]:
nb.score(X_train,y_train)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

DecisionTreeClassifier Modeli İle Eğitim

In [ ]:
dtc = DecisionTreeClassifier(random_state = 42, criterion='entropy')
dtc = dtc.fit(X_train, y_train)

In [ ]:
y_pred = dtc.predict(X_test)

DecisionTreeClassifier eğitim modelinin sonuçları:

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
cross_val_score(dtc, X, y, cv = 10).mean()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

KNN Modeli İle Eğitim

In [ ]:
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

KNN eğitim modelinin sonuçları:

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
score_list = []

for each in range(2,50,1):
    knn = KNeighborsClassifier(n_neighbors = each)
    knn.fit(X_train,y_train)
    score_list.append(knn.score(X_test, y_test))

plt.plot(range(2,50,1),score_list)
plt.xlabel("k en yakın komşu sayıları")
plt.ylabel("doğruluk skoru")
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score